In [ ]:
!pip install torch-geometric

In [ ]:
import os
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch_geometric as pyg
from torch_geometric import nn as gnn

from matplotlib import pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
splits = ["train", "valid", "test"]

layout_nlp_default = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default'
layout_nlp_random = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random'
layout_xla_default = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/default'
layout_xla_random = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/xla/random'

In [ ]:
def load_n_data_to_df(directory, split, n, pos=0):
    
    path = os.path.join(directory, split)
    files = os.listdir(path)
    
    data_list = []
    
    n = min(n, len(files))
    for file in tqdm(files[pos:pos + n]):
        file_path = os.path.join(path, file)
        model_graph = dict(np.load(file_path))
        model_graph["file"] = file
        data_list.append(model_graph)
    
    return pd.DataFrame(data_list)

In [ ]:
def load_data_to_df(directory, split):
    
    n = len(os.listdir(os.path.join(directory, split)))
    return load_n_data_to_df(directory, split, n)

In [ ]:
class LayoutLazyDataset(Dataset):
    
    def __init__(self, path, split):
        
        self.data_folder = os.path.join(path,split)
        self.data_files = os.listdir(self.data_folder)

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, index):
        data_file = self.data_files[index]
        
        # Load data sample from storage when it's actually needed
        data = self.load_data(data_file)

        file = data['file']
        node_feat = torch.from_numpy(data['node_feat'])
        node_opcode = torch.from_numpy(data['node_opcode']).type(torch.int64)
        edge_index = torch.from_numpy(data['edge_index']).permute(1, 0)
        node_config_feat = torch.from_numpy(data['node_config_feat'])
        node_config_ids = torch.from_numpy(data['node_config_ids'])
        config_runtime = torch.from_numpy(data['config_runtime']).type(torch.float32)
            
        return {
            "file": file,
            "node_feat": node_feat,
            "node_opcode": node_opcode,
            "edge_index": edge_index,
            "node_config_feat": node_config_feat,
            "node_config_ids": node_config_ids,
            "y": config_runtime
        }

    def load_data(self, data_file):
        
        data = dict(np.load(os.path.join(self.data_folder, data_file)))
        data["file"] = data_file
        
        return data


In [ ]:
class LayoutModel(nn.Module):
    
    def __init__(self, opcode_embedding_dim = 4, conv_hidden_channels = 120, conv_num_layers = 4, conv_out_dim = 48):
        super().__init__()
        
        self.opcode_embedding = nn.Embedding(120, opcode_embedding_dim)
        
        node_feature_dim = 140
        node_config_dim = 18
        
        node_rep_dim = opcode_embedding_dim + node_feature_dim + node_config_dim
        
        self.conv = gnn.GraphSAGE(node_rep_dim, conv_hidden_channels, conv_num_layers, conv_out_dim)
        
        hidden_dim = 48
        self.fwd = nn.Sequential(
            nn.Linear(conv_out_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, node_feat, node_opcode, edge_index, node_config_feat, node_config_ids):
        """
            Shapes:
                node_feat    - (n, 140)
                node_opcode  - (n, )
                edge_index   - (2, m)
                node_config_feat  - (c, nc, 18)
                node_config_ids - (nc, )
            
            Approach:
                1. Opcode embeddings
                2. Concatenate embeddings and config to node feature-vector
                3. Convolutional layer for node embeddings
                4. Pooling for graph embedding
                6. Forward layer
                7. Flatten        
        """
        n = node_feat.shape[0]
        c = node_config_feat.shape[0]
        
        node_opcode_embedding = self.opcode_embedding(node_opcode) # (n, opcode_embedding_dim)
        
        empty_configs = torch.full([n, 18], -1).to(device)
        
        x = torch.cat([node_opcode_embedding, node_feat, empty_configs], dim=1) # (n, opcode_embedding_dim + 140 + 18)

        runtimes = []
        
        # print(x.shape)
        x = x.unsqueeze(0)
        # print(x.shape)
        x = x.repeat(c, 1, 1)
        # print(x.shape)
        
        # print(node_config_feat.shape)
        for i, config in enumerate(node_config_feat):
        #    print(i)
            for node_id, config_feat in zip(node_config_ids, config):
                x[i, node_id, -18:] = config_feat
            
        x = self.conv(x, edge_index) # (c, n, conv_out_dim)
        # print(cx.shape)
        x = torch.mean(x, 1) # (c, conv_out_dim)
        # print(cx.shape)
        x = self.fwd(x) # (c, )
        return torch.flatten(x) # (c, )
        
    
        
        

In [ ]:
custom_dataset = LayoutLazyDataset(layout_nlp_random, 'train')

lazy_loader = DataLoader(custom_dataset, batch_size=1)

model = LayoutModel().to(device)

for data in custom_dataset:
    #print(data)
    for k, v in data.items():
        if isinstance(v, torch.Tensor):
            data[k] = v.to(device)
    
    node_feat, node_opcode, edge_index, node_config_feat, node_config_ids = data["node_feat"], data["node_opcode"], data["edge_index"], data["node_config_feat"], data["node_config_ids"]
    print(model(node_feat, node_opcode, edge_index, node_config_feat[0:2, :, :], node_config_ids).shape)
    break


In [ ]:
print(f"Total parameters in the model: {sum(p.numel() for p in model.parameters())}")

## Training

In [ ]:
def node_config_selection(node_config_feat, config_runtime, n):
    c = config_runtime.shape[0]

    num_chosen_configs = min(c, n)
    configs = list(range(0, c))
    random.shuffle(configs)
    chosen_configs = configs[:num_chosen_configs]
    return node_config_feat[chosen_configs, :, :], config_runtime[chosen_configs]

In [ ]:
def train_step(model, dataloader, criterion, opt):
    model.train()
    
    epoch_train_losses = np.empty((0,))
    for layout in tqdm(dataloader):
        
        node_feat, node_opcode, edge_index, node_config_feat, node_config_ids = layout["node_feat"][0].to(device), layout["node_opcode"][0].to(device), layout["edge_index"][0].to(device), layout["node_config_feat"][0].to(device), layout["node_config_ids"][0].to(device)
        y = layout["y"][0]
        
        node_config_feat, y = node_config_selection(node_config_feat, y, 100)
        opt.zero_grad()
        pred = model(node_feat, node_opcode, edge_index, node_config_feat, node_config_ids).to("cpu")
        loss = criterion(pred, y)
                        
        loss.backward()
        opt.step()
        
        epoch_train_losses = np.append(epoch_train_losses, loss.item())
    return np.mean(epoch_train_losses)

def test(model, dataloader, criterion):
    model.eval()
    
    epoch_valid_losses = np.empty((0,))
    
    with torch.no_grad():
        for layout in dataloader:
                
            node_feat, node_opcode, edge_index, node_config_feat, node_config_ids = layout["node_feat"][0].to(device), layout["node_opcode"][0].to(device), layout["edge_index"][0].to(device), layout["node_config_feat"][0].to(device), layout["node_config_ids"][0].to(device)
            y = layout["y"][0]
            
            node_config_feat, y = node_config_selection(node_config_feat, y, 100)
            
            pred = model(node_feat, node_opcode, edge_index, node_config_feat, node_config_ids).to("cpu")
            loss = criterion(pred, y)
            
            epoch_valid_losses = np.append(epoch_valid_losses, loss.item())
    return np.mean(epoch_valid_losses)

def train(model, trainloader, validloader, lr = 0.01, epochs = 10):
    
    model.to(device)
    
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    train_losses = np.empty((0,))
    valid_losses = np.empty((0,))
    
    for i in range(1, epochs + 1):
    
        train_loss = train_step(model, trainloader, loss_fn, optimizer)
        valid_loss = test(model, validloader, loss_fn)
    
        train_losses = np.append(train_losses, train_loss)
        valid_losses = np.append(valid_losses, valid_loss) 
        print(f"Epoch: {i}, Train Loss: {train_loss}, Valid Loss: {valid_loss}")
    
    return valid_losses[-1]

In [ ]:
model = LayoutModel()
trainloader = DataLoader(LayoutLazyDataset(layout_nlp_default, 'train'), batch_size = 1, shuffle = True)
validloader = DataLoader(LayoutLazyDataset(layout_nlp_default, 'valid'), batch_size = 1, shuffle = False)

In [ ]:
torch.manual_seed(42)
random.seed(42)
# train(model, trainloader, validloader)